In [0]:
!python --version

Python 3.9.5


In [0]:
from pyspark.sql.functions import *

In [0]:
hostName = "group10whserver"
port = 1433
database = "torontoWeatherKSI_warehouse"
user = "azureuser"
password = "Thien03@n1922"

In [0]:
table = spark.read.jdbc(
     url=f"jdbc:sqlserver://{hostName}.database.windows.net:{port};database={database};user={user};password={password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;driver=com.microsoft.sqlserver.jdbc.SQLServerDriver",
     table='TorontoKSIEvents_2012to2022')

display(table)

Date,Injury,Visibility,Light,RoadCondition,ImpactType
2012-02-08,Major,Clear,Daylight,Dry,Turning Movement
2012-02-08,Minimal,Clear,Daylight,Dry,Turning Movement
2012-02-08,null,Clear,"Dusk, artificial",Dry,Pedestrian Collisions
2012-02-08,None,Clear,"Dusk, artificial",Dry,Pedestrian Collisions
2012-02-08,Major,Clear,"Dusk, artificial",Dry,Pedestrian Collisions
2012-02-09,Minimal,Clear,Daylight,Dry,Rear End
2012-02-09,null,Clear,Daylight,Dry,Rear End
2012-02-09,None,Clear,Daylight,Dry,Rear End
2012-02-09,Major,Clear,Daylight,Dry,Rear End
2012-02-10,None,Drifting Snow,"Dusk, artificial",Dry,Pedestrian Collisions


In [0]:
table.createOrReplaceTempView("temp")
df = spark.sql("SELECT * FROM temp")
df_groupByInjury = df.groupby("Injury").agg(
    count(when((df.Light=="Daylight") | (df.Light=="DayLight, artificial"), df.Light)).alias("DayLight"), # Light
    count(when((df.Light=="Dark") | (df.Light=="Dark, artificial"), df.Light)).alias("Dark"),
    count(when((df.Light=="Dawn") | (df.Light=="Dawn, artificial"), df.Light)).alias("Dawn"),
    count(when((df.Light=="Dusk") | (df.Light=="Dusk, artificial"), df.Light)).alias("Dusk"),
    count(when(df.Light=="Other", df.Light)).alias("OtherLightCond"),
    count(when(df.RoadCondition=="Dry", df.RoadCondition)).alias("DryRoad"), # RoadCondition
    count(when(df.RoadCondition=="Ice", df.RoadCondition)).alias("IcyRoad"),
    count(when((df.RoadCondition=="Wet") | (df.RoadCondition=="Spilled liquid"), df.RoadCondition)).alias("WetRoad"),
    count(when((df.RoadCondition=="Slush") | (df.RoadCondition=="Loose Snow") | (df.RoadCondition=="Packed Snow"), df.RoadCondition)).alias("SnowyRoad"),
    count(when(df.RoadCondition=="Loose Sand or Gravel", df.RoadCondition)).alias("GravelRoad"),
    count(when(df.RoadCondition=="Other", df.RoadCondition)).alias("OtherRoadCond"),
    count(when(df.Visibility=="Clear", df.Visibility)).alias("ClearV"), # Visibility
    count(when((df.Visibility=="Rain") | (df.Visibility=="Freezing Rain"), df.Visibility)).alias("RainV"),
    count(when((df.Visibility=="Snow") | (df.Visibility=="Drifting Snow"), df.Visibility)).alias("SnowV"),
    count(when(df.Visibility=="Fog, Mist, Smoke, Dust", df.Visibility)).alias("FogV"),
    count(when(df.Visibility=="Strong wind", df.Visibility)).alias("WindV"),
    count(when(df.Visibility=="Other", df.Visibility)).alias("OtherV"),
    count(when(df.ImpactType=="Pedestrian Collisions", df.ImpactType)).alias("PedestrianColision"), # ImpactType
    count(when(df.ImpactType=="Turning Movement", df.ImpactType)).alias("Turning"),
    count(when(df.ImpactType=="Approaching", df.ImpactType)).alias("Approaching"),
    count(when(df.ImpactType=="Cyclist Collisions", df.ImpactType)).alias("CyclistColision"),
    count(when(df.ImpactType=="Angle", df.ImpactType)).alias("Angle"),
    count(when(df.ImpactType.like("%SMV%"), df.ImpactType)).alias("SMV"),
    count(when(df.ImpactType.like("%Rear%"), df.ImpactType)).alias("Rear"),
    count(when(df.ImpactType.like("%Side%"), df.ImpactType)).alias("Sideswipe"),
    count(when(df.ImpactType=="Other", df.ImpactType)).alias("OtherImpact")
)

display(df_groupByInjury)

Injury,DayLight,Dark,Dawn,Dusk,OtherLightCond,DryRoad,IcyRoad,WetRoad,SnowyRoad,GravelRoad,OtherRoadCond,ClearV,RainV,SnowV,FogV,WindV,OtherV,PedestrianColision,Turning,Approaching,CyclistColision,Angle,SMV,Rear,Sideswipe,OtherImpact
None,2403,1544,50,146,2,3466,6,644,48,3,25,3712,401,47,11,2,22,2031,634,95,570,222,134,345,113,57
Minor,446,338,11,14,0,656,13,121,16,1,3,708,77,25,0,0,1,134,185,139,13,117,86,104,26,9
null,293,159,4,19,1,385,0,84,3,0,4,418,46,3,6,0,3,151,62,17,51,41,86,47,8,14
Fatal,334,271,12,18,2,518,1,92,5,0,17,557,50,8,1,1,17,370,70,20,31,23,88,27,7,7
Minimal,362,257,8,12,0,544,2,94,8,1,0,586,49,11,3,0,0,117,195,51,24,78,27,128,25,2
Major,2022,1367,50,115,1,2950,9,580,42,6,15,3161,370,49,10,1,11,1428,539,153,478,180,430,264,79,51


In [0]:
df_groupByInjury.write.jdbc(
     url=f"jdbc:sqlserver://{hostName}.database.windows.net:{port};database={database};user={user};password={password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;driver=com.microsoft.sqlserver.jdbc.SQLServerDriver",
     table='TorontoKSIEvents_InjuryStats_2012to2022',
     mode='overwrite')